In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [ ]:
plot_all(406125434,
#          large=True,
#          bkspace_min=0.1,
#          abs_xlim='3p',
#          abs_offset=2,
#          abs_xlim=(1690, 1710),
#          abs_ylim='3%',
#          abs_ylim=(0.9975, 1.0025),
        )

### Run at startup

In [ ]:
from astropy.io import fits
from astronet.preprocess import generate_input_records
from astronet.preprocess import preprocess
from light_curve_util import keplersplinev2
from light_curve_util import tess_io
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


tce_table = None


def plot_all(tic,
             large=False,
             dpi=100,
             use_actual_preprocess=True,
             bkspace_min=0.5,
             abs_xlim=None,
             abs_offset=0,
             abs_ylim=None):
    tess_data_dir = f'/mnt/tess/lc'
    reports_dir = f'/mnt/tess/rpt/png'
    tces_file = '/mnt/tess/astronet/tces-v3.csv'

    b = plt.get_cmap('tab20')(0)
    b2 = plt.get_cmap('tab20')(1)
    o = plt.get_cmap('tab20')(2)
    o2 = plt.get_cmap('tab20')(3)
    g = plt.get_cmap('tab20')(4)
    r = plt.get_cmap('tab20')(6)
    n = plt.get_cmap('tab20')(8)
    k = plt.get_cmap('tab20')(14)

    fsize = (16, 9)
    
    global tce_table
    if tce_table is None:
        tce_table = pd.read_csv(tces_file, header=0)

    tce = tce_table[tce_table.tic_id == tic]
    period = tce.Period.values.item()
    epoc = tce.Epoc.values.item()
    duration = tce.Duration.values.item()

    print(f'Epoc: {epoc}\nPeriod: {period}\nDuration: {duration}')
    
    def config_abs_plot(title):
        plt.legend()
        plt.title(title)
        if abs_xlim:
            if abs_xlim == '3p':
                minx = min(td) + abs_offset * period
                maxx = minx + 3.5 * period
                plt.xlim(minx, maxx)
            else:
                plt.xlim(*abs_xlim)
        if abs_ylim:
            if abs_ylim == '3%':
                miny = np.percentile(fs[~np.isnan(fs)], 3)
                maxy = np.percentile(fs[~np.isnan(fs)], 97)
                plt.ylim(miny, maxy)
            else:
                plt.ylim(*abs_ylim)
    
    
    file_names = tess_io.tess_filenames(tic, tess_data_dir)
    f = fits.open(file_names)

    td = f[1].data["TIME"]
    fd = f[1].data["KSPSAP_FLUX"]
    fs = f[1].data["SAP_FLUX"]
    
    if large:
        plt.figure(figsize=fsize, dpi=dpi)
    else:
        plt.figure(figsize=fsize, dpi=dpi)
        plt.subplot(2, 3, 1)

    plt.plot(td, fs, '-', alpha=0.6, color=g, label='SAP')
    config_abs_plot('fits data')
    
    
    if large:
        fig = plt.figure(figsize=fsize, dpi=dpi)
    else:
        plt.subplot(2, 3, 2)

    ut, uf = preprocess.read_and_process_light_curve(tic, tess_data_dir, 'SAP_FLUX')

    input_mask = preprocess.get_spline_mask(ut, period, epoc, duration)
    sf, mdata = keplersplinev2.choosekeplersplinev2(
        ut, uf, input_mask=input_mask, verbose=True, return_metadata=True,
        bkspace_min=bkspace_min,
    )

    plt.plot(ut, uf, '-', alpha=0.6, color=g, label='SAP')
    plt.plot(ut[input_mask], sf[input_mask], 'x', markersize=3, color=k, label='spline (OOT)')
    plt.plot(ut[~input_mask], sf[~input_mask], 'o', markersize=3, color=o, label='spline')
    config_abs_plot(f'raw | bkspace: {mdata.bkspace}')
    
    
    if large:
        fig = plt.figure(figsize=fsize, dpi=dpi)
    else:
        plt.subplot(2, 3, 3)

    ut, nf, fm = preprocess.detrend_and_filter(
        tic, ut, uf, period, epoc, duration, bkspace_min)
    sft, sff, sfn = preprocess.phase_fold_and_sort_light_curve(ut, nf, period, epoc)

    plt.plot(sft, sff, 'o', markersize=3, alpha=0.6, c=o, label='spline')
    sff_filtered = np.where((sff > 1.5) | (sff < -0.5), 0, sff)
    if len(sff_filtered):
        fmt, fm, _ = preprocess.phase_fold_and_sort_light_curve(ut, fm, period, epoc)
        mask = np.where(fm, 1, min(sff_filtered))
        plt.plot(fmt, mask, '-', markersize=1, alpha=0.6, c=r, label='OOT')
        title = f'phase folded ({int(max(sfn))} folds) / filtered'
    else:
        title = f'phase folded ({int(max(sfn))} folds) | WARNING: filtering removed all data'
    plt.legend()
    plt.title(title)


    if not use_actual_preprocess:

        if large:
            fig = plt.figure(figsize=fsize, dpi=dpi)
        else:
            plt.subplot(2, 3, 4)
        sgv = preprocess.global_view(tic, sft, sff, period)
        plt.plot(sgv, 'o-', markersize=3, color=o, label='spline')
        plt.legend()
        plt.title('global view')


        if large:
            fig = plt.figure(figsize=fsize, dpi=dpi)
        else:
            plt.subplot(2, 3, 5)
        lvs = preprocess.local_view(tic, sft, sff, period, duration, new_binning=True)
        plt.plot(lvs, 'o-', markersize=3, c=o, label='spline robust')
        plt.legend()
        plt.title('local view')


        if large:
            fig = plt.figure(figsize=fsize, dpi=dpi)
        else:
            plt.subplot(2, 3, 6)
        lvs = preprocess.secondary_view(tic, sft, sff, period, duration)
        plt.plot(lvs, 'o-', markersize=3, color=o, label='spline robust')
        plt.legend()
        _ = plt.title('secondary view')
        
        
    else:
    
        generate_input_records.FLAGS = generate_input_records.parser.parse_args([
          '--tess_data_dir', '/mnt/tess/lc',
          '--output_dir', '/dev/null',
          '--input_tce_csv_file', tces_file,
        ])


        row = list(tce.iterrows())[0][1]

        try:
            ex = generate_input_records._process_tce(row, bkspace_min=bkspace_min)
            title = 'preprocess'
        except:
            title = 'old detrending'
            ex = generate_input_records._process_tce(row, True, bkspace_min=bkspace_min)


        if large:
            plt.figure(figsize=fsize, dpi=dpi)
        else:
            plt.subplot(2, 3, 4)
        plt.plot(ex.features.feature['global_view'].float_list.value, color=n)
        plt.title(f'{title} global')

        if large:
            plt.figure(figsize=fsize, dpi=dpi)
        else:
            plt.subplot(2, 3, 5)
        plt.plot(ex.features.feature['local_view'].float_list.value, color=n)
        plt.title(f'{title} local')

        if large:
            plt.figure(figsize=fsize, dpi=dpi)
        else:
            plt.subplot(2, 3, 6)
        plt.plot(ex.features.feature['secondary_view'].float_list.value, color=n)
        plt.title(f'{title} secondary')
    
    
    plt.figure(figsize=(12, 8), dpi=dpi)
    plt.axis('off')
    im = plt.imread(f'{reports_dir}/TIC{tic}.png')
    _ = plt.imshow(im)